In [1]:
import os, sys
import numpy as np
sys.path.append('../')

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
from onsstove.onsstove import OnSSTOVE
from onsstove.layer import RasterLayer, VectorLayer
from onsstove.raster import interpolate
import time

start = time.time()

## 1. Create an OnSSTOVE model

In [4]:
nepal = OnSSTOVE(project_crs=3857, cell_size=(1000, 1000))
output_directory = 'results'
nepal.output_directory = output_directory

## 2. Read the scenario data

In [5]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_social_specs.csv"
nepal.read_scenario_data(path, delimiter=',')

## 3. Read the cooking technologies data

In [6]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\OnSSTOVE cases\NP_test_file_tech_specs.csv"
nepal.read_tech_data(path, delimiter=',')

## 4. Add a country mask layer

In [7]:
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Administrative\npl_admbnda_nd_20201117_shp\npl_admbnda_adm0_nd_20201117.shp"
mask_layer = VectorLayer('admin', 'adm_0', layer_path=path)
os.makedirs(f'{output_directory}/admin/adm_0', exist_ok=True)
mask_layer.reproject(3857, f'{output_directory}/admin/adm_0')
nepal.mask_layer = mask_layer
# add admin_1 or 2 or 3 layer to gdf here

## 5. Add a population base layer

In [8]:
path = r"data\population_npl_2018-10-01.tif"
os.makedirs(f'{output_directory}/demographics/population', exist_ok=True)
population = RasterLayer('demographics', 'population', layer_path=path, resample='sum')
population.reproject(3857, f'{output_directory}/demographics/population', 1000, 1000)
population.mask(nepal.mask_layer.layer, f'{output_directory}/demographics/population')
nepal.base_layer = population
nepal.population_to_dataframe(population)

## 6. Calibrate population and urba/rural split

In [9]:
nepal.calibrate_pop()
nepal.calibrate_urban()

## 7. Calculate parameters of base fuel (Biomass)

### 7.1. Health costs

In [10]:
# Calculate relative risk for each disease for the base fuel
rr_0_alri, rr_0_copd, rr_0_ihd, rr_0_lc = nepal.base_fuel.relative_risk()

# Calculate the Population Atributable Factor (PAF) for each disease for the base fuel
paf_0_alri = nepal.base_fuel.paf(rr_0_alri, 1 - nepal.specs['clean_cooking_access'])
paf_0_copd = nepal.base_fuel.paf(rr_0_copd, 1 - nepal.specs['clean_cooking_access'])
paf_0_ihd  = nepal.base_fuel.paf(rr_0_ihd, 1 - nepal.specs['clean_cooking_access'])
paf_0_lc = nepal.base_fuel.paf(rr_0_lc, 1 - nepal.specs['clean_cooking_access'])

### 7.2. Carbon emissions and related costs

In [11]:
nepal.base_fuel.carb(nepal.specs, nepal.gdf)

### 7.3. Time for travelling and collecting fuel, and cooking

In [12]:
# paths to GIS layers
nepal.base_fuel.friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_walking_only_friction_surface\2020_walking_only_friction_surface.geotiff"
nepal.base_fuel.forest_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\Forest cover\Global Forest Cover Change (GFCC).tif"

# Calculate total time
nepal.base_fuel.total_time(nepal.specs, population, output_directory)

# Add results to the model
nepal.raster_to_dataframe(nepal.base_fuel.total_time_yr, name='base_fuel_time', method='read')

# nepal.raster_to_dataframe(nepal.base_fuel.travel_time, name='base_fuel_travel_time', method='read')

## 8. Add wealth index GIS data

In [13]:
path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Demand\Wealth Index\Wealth index 2011.tif"
wealth_index = RasterLayer('demographics', 'Wealth_index', layer_path=path,  resample='average')
os.makedirs(f'{output_directory}/demographics/Wealth_index', exist_ok=True)
interpolate(wealth_index.path)
wealth_index.align(population.path, f'{output_directory}/demographics/Wealth_index')

### 8.1. Calculate value of time

In [14]:
# Based on wealth index, minimum wage and a lower an upper range for cost of oportunity
nepal.get_value_of_time(wealth_index)

## 9. Read electricity network GIS layers

In [15]:
# Read MV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\Nepal_DL0\Nepal_DL0.shp"
mv_lines = VectorLayer('electricity', 'MV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/MV_line', exist_ok=True)
mv_lines.reproject(3857, f'{output_directory}/electricity/MV_line')

# Read HV lines
path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\QGIS\Power network\HV-network\HV_lines_georeferenced.shp"
hv_lines = VectorLayer('electricity', 'HV_line', layer_path=path)
os.makedirs(f'{output_directory}/electricity/HV_line', exist_ok=True)
hv_lines.reproject(3857, f'{output_directory}/electricity/HV_line')

### 9.1. Calculate distance to electricity infrastructure 

In [16]:
nepal.distance_to_electricity(hv_lines=hv_lines, mv_lines=mv_lines)

## 10. Add night time lights data

In [17]:
path = r"data\nighttime_lights.tif"
os.makedirs(f'{output_directory}/electricity/Night_lights', exist_ok=True)
ntl = RasterLayer('electricity', 'Night_lights', layer_path=path, resample='average')
ntl.align(population.path, f'{output_directory}/electricity/Night_lights')
ntl.layer[np.isnan(ntl.layer)] = 0 # hey lets make sure that we interpolate NaNs

nepal.raster_to_dataframe(ntl.layer, name='Night_lights', method='read')

## 11. Calibrate current electrified population

In [18]:
nepal.elec_current()

print('Calibrated electrified population fraction:', nepal.gdf['Elec_pop_calib'].sum() / nepal.gdf['Calibrated_pop'].sum())

We have identified the existence of transformers or MV lines as input data; therefore we proceed using those for the calibration
Calibrated electrified population fraction: 0.5201280449140765


## 12. Adding GIS data for LPG supply

In [19]:
nepal.techs['LPG'].lpg_path = r"C:\Users\camilorg\Box Sync\EGI Energy Systems\06 Projects\2021 Nepal Geospatial cooking\02 - work\GIS-data\Supply\LPG\Nepal_Gas_12Jun2021_Final4.shp"
nepal.techs['LPG'].friction_path = r"C:\Users\camilorg\Box Sync\Clean Cooking Energy\OnSSTOVE\data\2020_motorized_friction_surface\2020_motorized_friction_surface.geotiff"

nepal.techs['LPG'].add_travel_time(population, output_directory)

## 13. Calculate benefits and costs of each technology

In [20]:
names = ['Electricity', 'Collected_Traditional_Biomass', 'LPG']
techs = [nepal.techs[name] for name in names]

# Loop through each technology and calculate all benefits and costs
for tech in techs:
    print(f'Calculating health benefits for {tech.name}...')
    tech.morbidity(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    tech.mortality(nepal.specs, nepal.gdf, paf_0_alri, paf_0_copd, paf_0_lc, paf_0_ihd)
    print(f'Calculating carbon emissions benefits for {tech.name}...')
    tech.carbon_emissions(nepal.specs, nepal.gdf, nepal.base_fuel.carbon)
    print(f'Calculating time saved benefits for {tech.name}...')
    tech.time_saved(nepal.gdf, nepal.specs, population, output_directory)
    print(f'Calculating costs for {tech.name}...')
    tech.discounted_om(nepal.gdf, nepal.specs)
    tech.discounted_inv(nepal.gdf, nepal.specs)
    tech.discounted_meals(nepal.gdf, nepal.specs)
    tech.discount_fuel_cost(nepal.gdf, nepal.specs, nepal.rows, nepal.cols)
    tech.salvage(nepal.gdf, nepal.specs)
    print(f'Calculating net benefit for {tech.name}...\n')
    tech.net_benefit(nepal.gdf)

Calculating health benefits for electricity...
Calculating carbon emissions benefits for electricity...
Calculating time saved benefits for electricity...
Calculating costs for electricity...
Calculating net benefit for electricity...

Calculating health benefits for traditional_biomass_collected...
Calculating carbon emissions benefits for traditional_biomass_collected...
Calculating time saved benefits for traditional_biomass_collected...
Calculating costs for traditional_biomass_collected...
Calculating net benefit for traditional_biomass_collected...

Calculating health benefits for lpg...
Calculating carbon emissions benefits for lpg...
Calculating time saved benefits for lpg...
Calculating costs for lpg...
Calculating net benefit for lpg...



## 14. Getting the max benefit technology for each cell

In [21]:
nepal.maximum_net_benefit()

## 15. Printing the results

In [22]:
nepal.gdf

,geometry,Pop,Calibrated_pop,IsUrban,Households,base_fuel_time,value_of_time,HV_line_dist,MV_line_dist,Night_lights,...,benefits_electricity,net_benefit_electricity,costs_traditional_biomass_collected,benefits_traditional_biomass_collected,net_benefit_traditional_biomass_collected,costs_lpg,benefits_lpg,net_benefit_lpg,final_tech,maximum_net_benefit
0,POINT (9090104.530 3543192.836),31.171455,31.344224,0,5.498987,4745.471554,0.032189,58.872746,40.521599,0.000000,...,1338.153686,NaN,0.916498,0.0,-0.916498,1471.833688,1273.737326,-198.096362,traditional_biomass_collected,-0.916498
1,POINT (9090104.530 3536192.836),46.757183,47.016337,0,8.248480,5194.932033,0.032323,52.392746,33.837849,0.000000,...,2131.426611,NaN,1.374747,0.0,-1.374747,2190.039537,2034.802070,-155.237468,traditional_biomass_collected,-1.374747
2,POINT (9083104.530 3535192.836),5.517799,5.548382,0,0.973400,3991.061114,0.032684,54.708317,35.846897,0.000000,...,214.770211,NaN,0.162233,0.0,-0.162233,258.487731,203.367581,-55.120149,traditional_biomass_collected,-0.162233
3,POINT (9084104.530 3535192.836),15.585728,15.672112,0,2.749493,4032.672201,0.032658,54.203320,35.355339,0.000000,...,610.160057,NaN,0.458249,0.0,-0.458249,729.396677,577.951877,-151.444800,traditional_biomass_collected,-0.458249
4,POINT (9064104.530 3534192.836),15.585728,15.672112,0,2.749493,4724.034660,0.035454,64.536812,47.634022,0.000000,...,702.381190,NaN,0.458249,0.0,-0.458249,748.961096,670.173010,-78.788087,traditional_biomass_collected,-0.458249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96717,POINT (9724104.530 3044192.836),98.039602,98.582991,0,17.295262,9404.475230,0.054270,13.038404,1.414214,0.000000,...,10087.404111,8433.662039,2.882544,0.0,-2.882544,NaN,9884.803549,NaN,electricity,8433.662039
96718,POINT (9797104.530 3044192.836),536.486856,539.460361,1,94.642169,5351.900735,0.055515,17.029387,0.000000,0.000000,...,34920.410678,25870.895263,15.773695,0.0,-15.773695,NaN,33811.751149,NaN,electricity,25870.895263
96719,POINT (9798104.530 3044192.836),315.580504,317.329624,0,55.671864,5412.375916,0.055268,17.117243,1.000000,0.000000,...,20664.904241,15341.659879,9.278644,0.0,-9.278644,NaN,20012.751577,NaN,electricity,15341.659879
96720,POINT (9722104.530 3043192.836),642.881959,646.445163,1,113.411432,9671.425593,0.057113,11.000000,0.000000,0.891932,...,70649.173106,59804.974169,18.901905,0.0,-18.901905,NaN,69320.646225,NaN,electricity,59804.974169


In [23]:
nepal.gdf.groupby(['Current_elec', 'final_tech']).agg({'Calibrated_pop': lambda row: sum(row) / 1000000,
                                                       'maximum_net_benefit': lambda row: sum(row) / 1000000,
                                                       'Pop': 'count'})

Calibrated_pop  \
Current_elec final_tech                                      
0            lpg                                  1.793200   
             traditional_biomass_collected        0.453378   
1            electricity                         26.363422   

                                            maximum_net_benefit    Pop  
Current_elec final_tech                                                 
0            lpg                                      11.338660  32134  
             traditional_biomass_collected            -0.013284   5981  
1            electricity                            1146.064176  58607

In [24]:
nepal.gdf.groupby(['Current_elec']).agg({'Calibrated_pop': lambda row: sum(row) / 1000000,
                                         'Elec_pop_calib': lambda row: np.nansum(row) / 1000000})

,Calibrated_pop,Elec_pop_calib
Current_elec,,
0,2.246578,0.000000
1,26.363422,14.880863


## 16. Saving data to raster files

In [35]:
nepal.to_raster('final_tech')
nepal.to_raster('net_benefit_electricity')
nepal.to_raster('net_benefit_lpg')
nepal.to_raster('net_benefit_traditional_biomass_collected')
nepal.to_raster('maximum_net_benefit')

Layer saved in results\Output\final_tech.tif

Variable codes:
    traditional_biomass_purchased: 0
    traditional_biomass_collected: 1
    improved_biomass_purchased: 2
    improved_biomass_collected: 3
    lpg: 4
    biogas: 5
    electricity: 6

Layer saved in results\Output\net_benefit_electricity.tif

Layer saved in results\Output\net_benefit_lpg.tif

Layer saved in results\Output\net_benefit_traditional_biomass_collected.tif

Layer saved in results\Output\maximum_net_benefit.tif



In [36]:
nepal.to_raster('costs_lpg')

Layer saved in results\Output\costs_lpg.tif



In [26]:
end = time.time()

diff = end - start
print('Execution time:', str(str(int(diff//60))) + ' min ' + str(int((diff)%60)) + ' sec')

Execution time: 1 min 1 sec
